# Set cluster, accounting group, script, etc.
Some variables can either be set in this notebook, or in environment variables.

It is assumed that this notebook is running in an SSH agent with keys loaded (see the README).

In [ ]:
import os

cluster_name = os.environ.get('JOBSERVANT_CLUSTER') or 'cluster.name.here.please'

accounting_group = os.environ.get('JOBSERVANT_ACCOUNTING_GROUP') or 'accounting-group-here-please'
job_params = {
    'account': accounting_group,
    'time': '00:01:30',

    # Can ask for additional stuff, e.g.
    # 'mem': "1500000"
}

# This is what we will run on the cluster:
job_script = """
hostname
echo 'Hello World'
sleep 60
"""

# Imports

In [ ]:
from importlib import reload
import os
import sys

# Find jobservant
this_dir = os.path.dirname(os.path.realpath('.'))
sys.path.append(this_dir)

# Load / reload jobservant
import jobservant
import jobservant.cluster_account
import jobservant.jupyter.job_presenter
reload(jobservant)
reload(jobservant.cluster_account)
reload(jobservant.jupyter.job_presenter)

from jobservant.cluster_account import ClusterAccount
from jobservant.jupyter.job_presenter import JobPresenter

# Connect to cluster and submit job

In [ ]:
# Try log_level='debug' for lotsa output
# Set username='whomever' if your username isn't automatically detected
cluster_account = ClusterAccount(server=cluster_name, log_level='info')

job = cluster_account.submit_job(text=job_script, **job_params)

# Monitor progress of job
Runs in the background so other cells can run

In [ ]:
job_presenter = JobPresenter(job)
job_presenter.progress(include_output=True)

# Low-level job checking

In [ ]:
print(job.jobid)
print(job.work_directory)
print(job.status())
print(job.queue_status_hash())

# High-level job checking

In [ ]:
job_presenter.status_table()

# Check job output manually
Throws an error if the job isn't finished

In [ ]:
job.fetch_output()

# Other actions

```
# Fetch stderr output at end of job
error = job.fetch_error()

# Fetch contents of any file in work directory
foo_buffer = job.fetch_file('foo.txt')

# Create a file in the work directory
content = "I am content for the file hello.txt"
job.create_remote_file('hello.txt', content)

# Cancel job
job.cancel()

# Clean up (delete) remote job directory
job.cleanup()
```